In [1]:
from mx.elemwise_ops import quantize_elemwise_op
from mx.mx_ops import quantize_mx_op
from mx.specs import MxSpecs

In [ ]:

def mx_quant_dequant(input, mx_specs, axes=-1):
    dtype = input.dtype
    bf_in = quantize_elemwise_op(
        input.float(), mx_specs=mx_specs, round=mx_specs["round_output"]
    )
    qin_elem_format = mx_specs["w_elem_format"]
    qin = quantize_mx_op(
        bf_in,
        mx_specs,
        elem_format=qin_elem_format,
        scale_mode=mx_specs['W_scale_mode'],
        axes=[axes],
        round=mx_specs["round_mx_output"],
    )
    qin = qin.to(dtype)
    return qin

In [ ]:
mx_specs = MxSpecs()
mx_specs['custom_cuda'] = True
mx_specs['w_scale_mode'] = 'fp4_e2m1'
mx_specs['per_tensor'] = False